In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
# # 데이터 불러오기
# train = pd.read_csv('./data/train.csv', index_col=0)
# test = pd.read_csv('./data/test.csv', index_col=0)
# sample_submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

# # Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
# column_number = {}
# for i, column in enumerate(sample_submission.columns):
#     column_number[column] = i
    
# def to_number(x, dic):
#     return dic[x]

# train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# # 모델에 적용할 데이터 셋 준비 
# x = train.drop(columns=['type', 'type_num'], axis=1)
# y = train['type_num']

# x = x.drop(columns=['fiberID'], axis=1)
# test_x = test.drop(columns=['fiberID'],axis=1)

# x_name=x.columns

In [ ]:
#x = np.load('./data/x_percentile.npy')
x = np.load('./data/x_try.npy')

In [ ]:
sample_submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

y = np.load('./data/y.npy')
pred = np.load('./data/pred.npy')
yy =pd.read_csv('./data/yy.csv',header=None)

path='./data/column_name.txt'
with open(path,'r')  as f:
    col_name=f.read() 

x_name=col_name.split('\n')
try:
    del(x_name[20])
except:
    pass

In [ ]:
test_x = pred

In [ ]:
# y=np.array(y)

In [ ]:
# x=np.array(x)
#test_x = np.array(test_x)

In [ ]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV

x_train, x_test, y_train, y_test = train_test_split(x,y, stratify=y,train_size=0.8, shuffle=True ,random_state=0)
#x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8, shuffle=True ,random_state=0)


In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

scaler1=StandardScaler()
#scaler1=RobustScaler()
scaler1.fit(x_train)
x_train=scaler1.transform(x_train)
x_test=scaler1.transform(x_test)


scaler2=StandardScaler()
#scaler2=RobustScaler()
scaler2.fit(test_x)
test_x =scaler2.transform(test_x)

In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# 현재까지 가장 안정적인 모델

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.callbacks import EarlyStopping
from sklearn.externals.joblib import parallel_backend
from keras import regularizers

In [ ]:
def build_network(init_mode='uniform',dropout_rate=0.5,learn_rate=0.01,neurons=128,optimizers='adam'):
    
    inputs=Input(shape=(20,), name='input')
    
    ### 4개의 층
    
    # 5개의 층

    x=Dense(128, activation='elu', name='hidden1')(inputs)
    x=Dense(neurons, activation='elu', name='hidden2', kernel_initializer=init_mode,\
           kernel_regularizer=regularizers.l2(0.01))(x)
    x=Dense(neurons, activation='elu', name='hidden3', kernel_initializer=init_mode,\
           kernel_regularizer=regularizers.l2(0.01))(x)
    x=Dense(neurons, activation='elu', name='hidden4',kernel_initializer=init_mode,\
           kernel_regularizer=regularizers.l2(0.01))(x)       
    x=Dropout(dropout_rate)(x)
    x=Dense(neurons, activation='elu', name='hidden5',kernel_initializer=init_mode,\
           kernel_regularizer=regularizers.l2(0.01))(x)
    
    prediction = Dense(19, activation='softmax', name='output')(x)
    model = Model(inputs=inputs, outputs=prediction)
    
 
    model.compile(optimizer=optimizers, loss='categorical_crossentropy', metrics=['accuracy'])
                  
    return model


In [ ]:
def create_hyperparameters():
    batches=[128,256,512]
    dropout_rate = [0.25,0.5] #np.linspace(0.1, 0.5, 3)
    epochs = [100, 130]
    neurons = [128, 256]
    init_mode = [ 'glorot_normal', 'he_normal']
    
    #return{"batch_size":batches, "optimizer":optimizers}

    return{"batch_size":batches, "dropout_rate":dropout_rate, "epochs":epochs, \
            "neurons":neurons, "init_mode":init_mode}

from keras.wrappers.scikit_learn import KerasClassifier
model=KerasClassifier(build_fn=build_network, verbose=1)

hyperparameters= create_hyperparameters()

#print(hyperparameters)
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

search = GridSearchCV(estimator=model, param_grid=hyperparameters, n_jobs=-1, cv=4, verbose=1)
#search = RandomizedSearchCV(estimator=model, param_distributions=hyperparameters, n_jobs=-1, cv=4, verbose=1)

search.fit(x_train, y_train)
print(search.best_params_)